# Load the hourly historical data

In [27]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('../')

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os
from dotenv import load_dotenv, find_dotenv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# data directories
load_dotenv(find_dotenv(), verbose=True)
dir_project = os.getenv('PROJECT_ROOT')
dir_data_raw = os.path.join(dir_project, 'data/raw/')
dir_data_interim = os.path.join(dir_project, 'data/interim/')
dir_data_processed = os.path.join(dir_project, 'data/processed/')
dir_data_external = os.path.join(dir_project, 'data/external/')
dir_models = os.path.join(dir_project, 'models/')

In [29]:
all_dfs_hourly = pd.read_excel(os.path.join(dir_data_raw, 'DATA HYDRO IVADO Data VF 2017-2020YTD.xlsx'), sheet_name=None)

The original excel file has several sheets:

In [30]:
all_dfs_hourly.keys()

dict_keys(['Demand', 'Wind', 'Zonal Demand', 'Weather', 'HOEP Price', 'Intertie Flow', 'Generator Output', 'Hourly by Generator Name', 'key Generator name - fuel type'])

In [46]:
for df_key in all_dfs_hourly:
    print(df_key, len(all_dfs_hourly[df_key]))

Demand 30824
Wind 30828
Zonal Demand 30828
Weather 30816
HOEP Price 30820
Intertie Flow 30816
Generator Output 30816
Hourly by Generator Name 30816


# Sheet-wise preprocess

Some of the sheets have different time format, we unify them first:

- `Weather`

In [31]:
df_weather = all_dfs_hourly['Weather']

In [32]:
# Fusion date and hour
df_weather['Fusionné'] = df_weather['Date'].dt.strftime('%m/%d/%Y') + ' ' + (df_weather['Hour']-1).astype(str) + ':00'
df_weather['Fusionné'] = pd.to_datetime(df_weather['Fusionné'])

In [33]:
df_weather.head()

,Date,Hour,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Stn Press (kPa),Fusionné
0,2017-01-01,1,0.5,-4.3,70.0,99.34,2017-01-01 00:00:00
1,2017-01-01,2,0.8,-4.2,70.0,99.52,2017-01-01 01:00:00
2,2017-01-01,3,0.6,-4.9,67.0,99.64,2017-01-01 02:00:00
3,2017-01-01,4,1.0,-4.6,66.0,99.82,2017-01-01 03:00:00
4,2017-01-01,5,1.0,-4.6,66.0,99.98,2017-01-01 04:00:00


- `Intertie Flow` & `Hourly by Generator Name`

Similar to the above:

In [34]:
sheet = 'Intertie Flow'

df_temp = all_dfs_hourly[sheet]
df_temp['Fusionné'] = df_temp['date'].dt.strftime('%m/%d/%Y') + ' ' + (df_temp['hour'].astype(int)-1).astype(str) + ':00'
df_temp['Fusionné'] = pd.to_datetime(df_temp['Fusionné'])

In [35]:
sheet = 'Hourly by Generator Name'

df_temp = all_dfs_hourly[sheet]
df_temp['Fusionné'] = df_temp['Date'].dt.strftime('%m/%d/%Y') + ' ' + (df_temp['Hour']-1).astype(str) + ':00'
df_temp['Fusionné'] = pd.to_datetime(df_temp['Fusionné'])

- Unify the col name for `Wind`

In [36]:
df_wind = all_dfs_hourly['Wind']

df_wind.rename({'Date': 'Fusionné'}, axis='columns', inplace=True)

In [37]:
# clarify the col name
df_wind.rename({'Réalisé': 'wind_realized', 'Prévisionnel': 'wind_provisional'}, axis='columns', inplace=True)

- Drop duplicated `Ontario Demand` from one of the `Demand` and `Zonal Demand` sheets.

In [38]:
df_demand = all_dfs_hourly['Demand']
df_demand.drop(columns=['Ontario Demand'], inplace=True)

# Concatenate different sheets (variables)

In [39]:
all_dfs_hourly.pop('key Generator name - fuel type')

,Generator,Fuel Type
0,ABKENORA,HYDRO
1,ADELAIDE,WIND
2,AGUASABON,HYDRO
3,ALEXANDER,HYDRO
4,AMARANTH,WIND
...,...,...
179,NAPANEE-G3,GAS
180,NaN,NaN
181,Generator,Fuel Type
182,CRYSLER,WIND


In [40]:
from functools import reduce

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Fusionné'], 
                                                how='outer'), all_dfs_hourly.values())

In [41]:
df_merged.rename({'Fusionné': 'date'}, axis='columns', inplace=True)

In [42]:
len(df_merged.columns)

219

In [47]:
len(df_merged)

35424

Output:

In [43]:
df_merged.to_csv(os.path.join(dir_data_interim, 'hourly_data_merged_all_variables.csv'), index=False)